In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#================================================================#
#============ Preparacion de datos modelo baseline ==============#
#================================================================#

#====================== Import de librerias =====================#

import random
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import datetime
import plotly.express as px
import plotly.graph_objects as go
from datetime import date
import glob
import numpy as np
import torch
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp
from typing import Tuple, Dict, Any, List
from tqdm import tqdm, trange
from IPython import embed
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn import preprocessing


# Definicion de hiperparametros
hparams = {
    'batch_size':64,
    'num_epochs':12,
    'hidden_size': 32,
    'learning_rate':1e-4,
}

# we select to work on GPU if it is available in the machine, otherwise will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Nombre de columnas
col_names = {"col_id_product": "asin",
             "col_id_reviewer": "reviewerID",
             "col_unix_time": "timestamp",
             "col_year": "year",
             "col_rating": "overall"}

# Path de datos
path_data= "/content/drive/MyDrive/postgraduate/Trabajo_final/data"

# Numero de zero positions 
num_samples=199

#============ Definicion de funciones de extraccion y tratamiento ============#

def load_src_data(archivo):

  '''
  Funcion que importa datasets. 
  '''

  ### Import de información
  df = pd.read_pickle(archivo)

  return df

def preprocess_data(df, meta):  
  # Preprocesamos el dataset
  # Nos quedamos solo con usuario producto y time stamp
  # Pasamos ids a numerico
  # Los ids de producto y usuario no pueden ser los mismos, los transformamos para que el minimo id de producto se +1 el maximo de usuario

  data = df[[col_names["col_id_reviewer"], col_names["col_id_product"],  col_names["col_rating"],  col_names["col_unix_time"]]]

  # Aplicamos una lambda a la columna de rating para transformar los valores a 0 o 1
  data.loc[:, col_names["col_rating"]] = data[col_names["col_rating"]].apply(lambda x: 1 if x > 0 else 0)

  # Convertimos la columna de tiempo a formato fecha
  data.loc[:, col_names["col_unix_time"]] = pd.to_datetime(data[col_names["col_unix_time"]])

  # Codificación de etiquetas para el usuario
  le_usuario = preprocessing.LabelEncoder()
  le_usuario.fit(data[col_names["col_id_reviewer"]])
  data.loc[:, col_names["col_id_reviewer"]] = le_usuario.transform(data[col_names["col_id_reviewer"]]).astype('int32')

  # Codificación de etiquetas para el producto
  le_producto = preprocessing.LabelEncoder()
  le_producto.fit(data[col_names["col_id_product"]])
  data.loc[:, col_names["col_id_product"]] = le_producto.transform(data[col_names["col_id_product"]]).astype('int32')
  meta.loc[:, col_names["col_id_product"]] = le_producto.transform(meta[col_names["col_id_product"]]).astype('int32')

  add_dims = 0

  # Convertimos la data a formato numpy
  data = data.to_numpy()

  for i in range(data.shape[1] - 2):  # do not affect to timestamp
      # Hacemos que los valores empiecen desde 0
      data[:, i] -= np.min(data[:, i])
      # Re-indexamos
      data[:, i] += add_dims
      add_dims = np.max(data[:, i]) + 1

  dims_usuarios_productos = np.max(data[:,:2], axis=0) + 1

  meta.loc[:, col_names["col_id_product"]] = meta[col_names["col_id_product"]].apply(lambda x: x + dims_usuarios_productos[0]).astype('int32')


  print( "\n","Minimo id de usuario: ", np.min(data[:,0]), "\n",
        "Maximo id de usuario: ", np.max(data[:,0]), "\n",
        "Minimo id de producto: ", np.min(data[:,1]), "\n",
        "Maximo id de producto: ", np.max(data[:,1]), "\n",)
  
  return (data, meta, dims_usuarios_productos)

def build_adj_mx(n_feat:int, data:np.ndarray) -> sp.dok_matrix :
    """
    Esta funcion construye una matriz de adyacencia a partir de los datos de entrada.
    La matriz de adyacencia es una representacion simetrica de las interacciones entre los usuarios y productos
    y también las interacciones entre productos y cualquier información adicional.

    :param n_feat: El número de características presentes en los datos de entrada (número de columnas)
    :param data: La matriz de datos de entrada, donde cada fila representa una interacción entre un usuario y un producto.
    :return: La matriz de adyacencia, representada como un objeto dok_matrix, que es una estructura eficiente para construir matrices sparse.
    """

    # Instanciamos el objeto train_mat como una dok_matrix
    # Una estructura eficiente para construir matrices sparse
    train_mat = sp.dok_matrix((n_feat, n_feat), dtype=np.float32)
    for x in tqdm(data, desc=f"BUILDING ADJACENCY MATRIX..."):
        # rellanamos la matriz, al ser simetrica rellenamos primero con logica usuario-producto y luego con logica producto-usuario
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0
        # IDEA: Tratamos las caracteristicas que no son usuario o producto de forma diferente porque no consideramos
        #  las interacciones entre contextos
        # Añadimos informacion extra a parte de la interacion usuario producto (aqui podria ir el rating)
        if data.shape[1] > 2:
            for idx in range(len(x[2:])):
                train_mat[x[0], x[2 + idx]] = 1.0
                train_mat[x[1], x[2 + idx]] = 1.0
                train_mat[x[2 + idx], x[0]] = 1.0
                train_mat[x[2 + idx], x[1]] = 1.0

    return train_mat


def ng_sample(data: np.ndarray, dims: list, num_ng:int=4) -> Tuple[np.ndarray, sp.dok_matrix]:
  """
  Crea una matriz de interacciones de usuario-producto (rating) y aplica un muestreo negativo.
  
  Args:
  data: np.ndarray, con las interacciones usuario-producto.
  dims: lista, con los valores mínimo y máximo de los productos y usuarios.
  num_ng: int, número de interacciones negativas que se quieren generar por cada interacción positiva.
  
  Returns:
  np.ndarray, con las interacciones usuario-producto incluyendo las interacciones negativas generadas.
  sp.dok_matrix, con la matriz de interacciones usuario-producto.
  """

  # Creación de la matriz de interacciones positivas
  rating_mat = build_adj_mx(dims[-1], data)
  interactions = []
  min_item, max_item = dims[0], dims[1]
  for num, x in tqdm(enumerate(data), desc='Performando muestreo negativo...'):
      # Añade la interacción positiva al arreglo de interacciones
      interactions.append(np.append(x, 1))
      for t in range(num_ng):
          # Selecciona un producto al azar para generar una interacción negativa
          j = np.random.randint(min_item, max_item)
          # Verifica que la interacción no sea una interacción positiva previa
          while (x[0], j) in rating_mat or j == int(x[1]):
              j = np.random.randint(min_item, max_item)
          # Añade la interacción negativa al arreglo de interacciones
          interactions.append(np.concatenate([[x[0], j], x[2:], [0]]))
  # Devuelve la matriz con todas las interacciones y la matriz dispersa con las interacciones positivas
  return np.vstack(interactions), rating_mat


def create_test_no_interactions(train_x: np.ndarray, test_x: np.ndarray, dims_usuarios_productos: Tuple[int, int],  num_samples: int) -> np.ndarray:
    """
    Esta funcion se encarga de crear de manera eficiente un dataset que contenga las interacciones usuario-producto en test que no se hayan producido.
    
    Argumentos:
        train_x (np.ndarray): matriz de entrenamiento con las interacciones usuario-producto previas
        test_x (np.ndarray): matriz de prueba con las interacciones usuario-producto previas
        dims_usuarios_productos (Tuple[int, int]): rango de productos y usuarios disponibles
    
    Retorno:
        np.ndarray: una matriz con todas las interacciones usuario-producto en test que no se hayan producido
    """
    from tqdm import tqdm
    import random
    
    # Identificamos los usuarios presentes en la prueba
    usuarios_test = np.unique(test_x[:, 0])
    # Identificamos el rango de productos disponibles
    total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])
    
    # Recorremos cada usuario presente en la prueba
    for usuario in tqdm(usuarios_test):
        # Identificamos los productos en los que el usuario ha interactuado previamente en entrenamiento
        productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
        # Seleccionamos al azar 199 productos con los que el usuario no ha interactuado previamente
        productos_a_machear = random.choices(list(set(total_productos) - set(productos_train)), k=num_samples)
        # Creamos una lista de interacciones usuario-producto para este usuario
        lista_por_usuario = [[usuario, x] for x in productos_a_machear]
        
        # Si es el primer usuario, inicializamos una matriz con sus interacciones
        if usuario == 0:
            zero_positions = np.array(lista_por_usuario)
        # Si no es el primer usuario, concatenamos sus interacciones a la matriz existente
        else:
            zero_positions = np.concatenate((zero_positions, np.array(lista_por_usuario)), axis=0)
            
    return zero_positions


def build_test_set(itemsnoninteracted:list,
                   gt_test_interactions: np.ndarray) -> list:
    """
    Construye el conjunto de pruebas para un usuario dado
    :param itemsnoninteracted: lista de items que no han sido interactuados por el usuario
    :param gt_test_interactions: arreglo numpy con las interacciones verdaderas del usuario para pruebas
    :return: lista de arreglos numpy con interacciones positivas y negativas para cada usuario
    """
    # max_users, max_items = dims 
    test_set = []
    # Recorre cada par de interacciones verdaderas y items no interactuados
    for pair, negatives in tqdm(zip(gt_test_interactions, itemsnoninteracted), desc="BUILDING TEST SET..."):
        # AGREGA EL CONJUNTO DE PRUEBAS PARA UN SOLO USUARIO
        # Elimina el item que si fue interactuado
        negatives = np.delete(negatives, np.where(negatives == pair[1]))
        # Crea una matriz con la interacción verdadera y items no interactuados
        single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
        single_user_test_set[:, 1][1:] = negatives
        test_set.append(single_user_test_set.copy())
    return test_set



class PointData(Dataset):
    def __init__(self,
                 data: np.ndarray,
                 dims: list) -> None:
        """
        Dataset formatter adapted point-wise algorithms
        Parameters
        """
        super(PointData, self).__init__()
        self.interactions = data
        self.dims = dims

    def __len__(self) -> int:
        return len(self.interactions)
        
    def __getitem__(self, index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Return the pairs user-item and the target.
        """
        return self.interactions[index][:-1], self.interactions[index][-1]

def split_train_test(data: np.ndarray,n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        # Filtramos al usuario dentro del dataset
        user_data = data[data[:, 0] == u]
        # Ordenamos por fecha de interaccion
        sorted_data = user_data[user_data[:, 2].argsort()]

        # Si solo existe una review, introduce esta interaccion en el train
        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:-1])

        else:
          # Si la ultima interaccion tiene score de 1 puede ir a test, si no, todo a train
          if sorted_data[-1][-2]==1:
            # Introduce todas las interacciones salvo la ultima en el train
            train_x.append(sorted_data[:-1][:, :-1])
            # La ultima interaccion corresponde al test
            test_x.append(sorted_data[-1][:-1])

          else:
            train_x.append(sorted_data[:, :-1])

    return np.vstack(train_x), np.stack(test_x)


#============ Definicion de modelado ============#

#============ Definicion de visualizacion ============#

In [3]:
# Lectura de la información
file_to_src = glob.glob(path_data+'/SRC*.pkl')
df=load_src_data(file_to_src[0])# Reviews
meta=load_src_data(file_to_src[1])# Metadatos

In [4]:
# Preprocesamiento de reviews y metadatos
data, meta, dims_usuarios_productos = preprocess_data(df, meta)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)



 Minimo id de usuario:  0 
 Maximo id de usuario:  10758 
 Minimo id de producto:  10759 
 Maximo id de producto:  14116 



In [5]:
# Division entre train y test
train_x, test_x = split_train_test(data, dims_usuarios_productos[0])

spliting train/test and removing timestamp...: 100%|██████████| 10759/10759 [00:19<00:00, 549.59it/s]


In [6]:
train_x=train_x.astype(int)
test_x=test_x.astype(int)

In [7]:
# Eliminamos timestamp
data=data[:,:-1]

In [8]:
# realizamos el negative sampling
train_x, rating_mat = ng_sample(train_x[:, :2], dims_usuarios_productos)

BUILDING ADJACENCY MATRIX...: 100%|██████████| 91008/91008 [00:01<00:00, 46824.26it/s]
Performando muestreo negativo...: 91008it [00:08, 10542.77it/s]


In [9]:
train_x

array([[    0, 10824,     1],
       [    0, 11656,     0],
       [    0, 13361,     0],
       ...,
       [10758, 12635,     0],
       [10758, 13297,     0],
       [10758, 14106,     0]])

In [10]:
# Creamos el dataset de entrenamiento. Devuelve array con interaccion usuario-producto y si sucedió de verdad
train_dataset = PointData(train_x, dims_usuarios_productos)
train_dataset[1]

(array([    0, 11656]), 0)

In [14]:
# Se crean interacciones que no se han dado (0's) en datos de test
zero_positions=create_test_no_interactions(train_x,test_x, dims_usuarios_productos, num_samples )

100%|██████████| 10759/10759 [01:21<00:00, 132.05it/s]


In [15]:
zero_positions

array([[    0, 12227],
       [    0, 12919],
       [    0, 11297],
       ...,
       [10758, 13412],
       [10758, 11182],
       [10758, 13317]])

In [16]:
# Se crea una lista de listas con los productos por usuario a testear
items2compute = []
for user in trange(dims_usuarios_productos[0]):
    aux = zero_positions[zero_positions[:, 0] == user][:, 1]
    items2compute.append(aux[aux >= dims_usuarios_productos[0]])

100%|██████████| 10759/10759 [01:17<00:00, 139.57it/s]


In [17]:
print(test_x)

[[    0 12806     1]
 [    1 14064     1]
 [    2 12659     1]
 ...
 [10756 12639     1]
 [10757 11148     1]
 [10758 14067     1]]


In [18]:
# Al estas en esta tabla todas las interacciones positivas, no necesito mas a esa ultima columna
test_x = test_x[:, :2]
test_x

array([[    0, 12806],
       [    1, 14064],
       [    2, 12659],
       ...,
       [10756, 12639],
       [10757, 11148],
       [10758, 14067]])

In [19]:
# Se construye el test_set definitivo, con las interacciones que han ocurrido y las que no en test set
test_x = build_test_set(items2compute, test_x)

BUILDING TEST SET...: 10759it [00:02, 5348.72it/s]


In [21]:
train_dataset.dims

array([10759, 14117], dtype=object)

In [ ]:
test_x[0]

In [ ]:
# Guarda train_dataset y test_x
import pickle
from datetime import date

today = date.today()

# Train
with open(path_data+"/"+"MOD_baseline_train"+"_"+today.strftime("%d%m%Y")+".pkl", 'wb') as handle:
    pickle.dump(train_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Test
with open(path_data+"/"+"MOD_baseline_test"+"_"+today.strftime("%d%m%Y")+".pkl", 'wb') as f:
    pickle.dump(test_x, f)

# Meta
meta.to_pickle(path_data+"/"+"SRC_meta_Musical_instruments_id_treated"+"_"+today.strftime("%d%m%Y")+".pkl")